In [1]:
import pandas as pd
import os
import re
from pathlib import Path

# 1️⃣ 폴더 경로 설정
base_path = r"C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종"

# 2️⃣ 결과 저장 경로
output_path = os.path.join(base_path, "BM_통합파일.xlsx")

# 3️⃣ 대상 파일 패턴: "YYYYMM_사학연금_포트폴리오_분석.xlsm"
pattern = re.compile(r"^(20\d{2}(0[1-9]|1[0-2]))_사학연금_포트폴리오_분석\.xlsm$", re.IGNORECASE)

# 4️⃣ 폴더 내 대상 파일 수집
files = []
for f in os.listdir(base_path):
    m = pattern.match(f)
    if m:
        yyyymm = m.group(1)
        files.append((yyyymm, os.path.join(base_path, f), f))

# 정렬(월 기준)
files.sort(key=lambda x: x[0])

if not files:
    print("⚠️ 대상 파일이 없습니다. (예: 202512_사학연금_포트폴리오_분석.xlsm)")
else:
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        for yyyymm, file_path, file_name in files:
            try:
                df = pd.read_excel(
                    file_path,
                    sheet_name="BM",
                    usecols="AB:AK"
                )

                # 시트명 = YYYYMM
                sheet_name = yyyymm

                # 엑셀 시트명 제한(31자) 대응(안전장치)
                sheet_name = sheet_name[:31]

                df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"✅ {file_name} -> {sheet_name} 시트 저장 완료")

            except Exception as e:
                print(f"❌ 처리 실패: {file_name} / 사유: {e}")

    print(f"\n📘 통합 파일 생성 완료: {output_path}")


✅ 202510_사학연금_포트폴리오_분석.xlsm -> 202510 시트 저장 완료
✅ 202511_사학연금_포트폴리오_분석.xlsm -> 202511 시트 저장 완료
✅ 202512_사학연금_포트폴리오_분석.xlsm -> 202512 시트 저장 완료

📘 통합 파일 생성 완료: C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\BM_통합파일.xlsx


In [2]:
import pandas as pd
import os
import re

# =========================
# 1️⃣ 원본 파일 (BM_통합파일_추가)
# =========================
base_path = r"C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종"

pattern = re.compile(r"^BM_통합파일_추가(_.*)?\.xlsx$", re.IGNORECASE)
matched_files = [f for f in os.listdir(base_path) if pattern.match(f)]

if not matched_files:
    raise FileNotFoundError("BM_통합파일_추가*.xlsx 파일을 찾을 수 없습니다.")

# 최신 파일 선택
matched_files.sort(
    key=lambda f: os.path.getmtime(os.path.join(base_path, f)),
    reverse=True
)
input_path = os.path.join(base_path, matched_files[0])
print(f"📂 입력 파일: {input_path}")

# =========================
# 2️⃣ 출력 파일
# =========================
output_path = os.path.join(base_path, "BM_컬럼제거_통합.xlsx")

# =========================
# 3️⃣ 모든 시트 로드
# =========================
excel_file = pd.ExcelFile(input_path)
all_sheets = excel_file.sheet_names

# YYYYMM 시트만 대상
target_sheets = [s for s in all_sheets if len(s) == 6 and s.isdigit()]
target_sheets.sort()
print("처리 대상 시트:", target_sheets)

# =========================
# 4️⃣ 시트별 처리
# =========================
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    for sheet in target_sheets:
        df = pd.read_excel(input_path, sheet_name=sheet, header=None)

        # ✅ 1) 첫 행 제거
        df = df.iloc[1:].reset_index(drop=True)

        # (안전장치) 첫 행 제거 후 비어있으면 스킵
        if df.empty:
            print(f"⚠️ {sheet}: 첫 행 제거 후 데이터가 없어 스킵")
            continue

        # ✅ 2) (제거 후) 새 첫 행 기준으로 'EQUITY_RETURN' 포함 컬럼 제거
        equity_cols = [
            col for col in df.columns
            if "EQUITY_RETURN" in str(df.iloc[0, col])
        ]

        if equity_cols:
            df = df.drop(columns=equity_cols)
            print(f"🧹 {sheet}: EQUITY_RETURN 컬럼 {len(equity_cols)}개 제거")
        else:
            print(f"ℹ️ {sheet}: 제거 대상 없음")

        # ✅ 3) 저장 (첫 행 제거된 상태로 저장)
        df.to_excel(writer, sheet_name=sheet, index=False, header=False)

print(f"\n✅ 첫 행 제거 + 컬럼 제거 완료 → {output_path}")


📂 입력 파일: C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\BM_통합파일_추가.xlsx
처리 대상 시트: ['202510', '202511', '202512']
🧹 202510: EQUITY_RETURN 컬럼 4개 제거
🧹 202511: EQUITY_RETURN 컬럼 4개 제거
🧹 202512: EQUITY_RETURN 컬럼 4개 제거

✅ 첫 행 제거 + 컬럼 제거 완료 → C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\BM_컬럼제거_통합.xlsx


In [3]:
import pandas as pd
import os
import re

# =========================
# 1️⃣ 입력 파일 자동 선택 (BM_컬럼제거_통합*.xlsx)
# =========================
base_path = r"C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종"

pattern = re.compile(r"^BM_컬럼제거_통합(_.*)?\.xlsx$", re.IGNORECASE)
matched_files = [f for f in os.listdir(base_path) if pattern.match(f)]

if not matched_files:
    raise FileNotFoundError("BM_컬럼제거_통합*.xlsx 파일을 찾을 수 없습니다.")

# 최신 파일 선택
matched_files.sort(key=lambda f: os.path.getmtime(os.path.join(base_path, f)), reverse=True)
file_path = os.path.join(base_path, matched_files[0])
print(f"📂 처리 대상 파일: {file_path}")

# =========================
# 2️⃣ 모든 시트 불러오기
# =========================
excel_file = pd.ExcelFile(file_path)
all_sheets = excel_file.sheet_names

# YYYYMM 시트만 대상 (파일에 존재하는 것 전부)
target_sheets = [s for s in all_sheets if len(s) == 6 and s.isdigit()]
target_sheets.sort()
print("처리할 시트들:", target_sheets)

# =========================
# 3️⃣ 결과 저장 리스트
# =========================
final_data = []

# =========================
# 4️⃣ 시트별 변환
# =========================
for sheet in target_sheets:
    df_raw = pd.read_excel(file_path, sheet_name=sheet, header=None)

    if df_raw.empty:
        continue

    # ① 컬럼명 세팅(첫 번째 행을 컬럼명으로 설정)
    df_raw.columns = df_raw.iloc[0]
    df = df_raw.iloc[1:].reset_index(drop=True)

    # ② 여기에서 A열 기준 빈 행 삭제 ← ⭐ 이 위치가 정답
    first_col = df.columns[0]
    df = df.dropna(subset=[first_col])

    
    # '국가' 컬럼 존재 여부 확인
    if '국가' not in df.columns:
        print(f"[⚠️ 경고] 시트 {sheet}에 '국가' 컬럼이 없습니다. 건너뜁니다.")
        continue

    # valid_columns 자동 생성: '국가' 제외 + NaN 컬럼 제외
    valid_columns = [c for c in df.columns if pd.notna(c) and c != '국가']

    # 기준일(동적): YYYYMM01
    기준일 = f"{sheet}01"

    for col in valid_columns:
        # 수치형 변환(비중이 문자열로 들어오는 경우 대비)
        w = pd.to_numeric(df[col], errors="coerce")

        temp = pd.DataFrame({
            '기준일': 기준일,
            '구분': 'FUND',
            '기관코드': '1061',
            '펀드코드': '',
            'BM코드': '',
            '자산구분(ASSET_ID)': col,
            '데이터ID(DATA_ID)': df['국가'],
            '데이터구분': 'BM',
            '비중(WGT)': w
        })

        final_data.append(temp)

# =========================
# 5️⃣ 모든 시트 통합
# =========================
if not final_data:
    raise ValueError("변환된 데이터가 없습니다. (국가 컬럼 누락/시트 없음/컬럼 구조 확인 필요)")

result_df = pd.concat(final_data, ignore_index=True)

# =========================
# 6️⃣ 비중 컬럼 정리
# =========================
result_df = result_df.dropna(subset=['비중(WGT)'])
result_df = result_df[result_df['비중(WGT)'] != 0]

# =========================
# 7️⃣ 결과 미리보기
# =========================
print(result_df.head())

# =========================
# 8️⃣ 저장 (파일명 축약)
# =========================
if not target_sheets:
    out_name = "BM 통합_데이터 변환.xlsx"
else:
    start = target_sheets[0]
    end = target_sheets[-1]

    sy, sm = start[:4], start[4:]
    ey, em = end[:4], end[4:]

    if sy == ey:
        # 같은 연도
        if sm == em:
            out_suffix = f"{sy}.{sm}"
        else:
            out_suffix = f"{sy}.{sm}~{em}"
    else:
        # 연도 넘어가는 경우
        out_suffix = f"{sy}.{sm}~{ey}.{em}"

    out_name = f"BM 통합_데이터 변환_{out_suffix}.xlsx"

output_path = os.path.join(base_path, out_name)
result_df.to_excel(output_path, index=False)

print(f"✅ {out_name} 저장 완료: {output_path}")


📂 처리 대상 파일: C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\BM_컬럼제거_통합.xlsx
처리할 시트들: ['202510', '202511', '202512']
        기준일    구분  기관코드 펀드코드 BM코드    자산구분(ASSET_ID)          데이터ID(DATA_ID)  \
0  20251001  FUND  1061            MSCI ACWI NR USD              Financials   
1  20251001  FUND  1061            MSCI ACWI NR USD  Information Technology   
2  20251001  FUND  1061            MSCI ACWI NR USD  Consumer Discretionary   
3  20251001  FUND  1061            MSCI ACWI NR USD             Health Care   
4  20251001  FUND  1061            MSCI ACWI NR USD             Industrials   

  데이터구분  비중(WGT)  
0    BM    16.81  
1    BM    28.51  
2    BM    10.52  
3    BM     8.56  
4    BM    10.61  
✅ BM 통합_데이터 변환_2025.10~12.xlsx 저장 완료: C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\BM 통합_데이터 변환_2025.10~12.xlsx


In [5]:
import pandas as pd
import os
import re

# =========================
# 1️⃣ 기존 변환된 파일 자동 선택
# =========================
base_path = r"C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종"

# BM 통합_데이터 변환_*.xlsx 탐색
pattern = re.compile(r"^BM 통합_데이터 변환_.*\.xlsx$")
matched_files = [f for f in os.listdir(base_path) if pattern.match(f)]

if not matched_files:
    raise FileNotFoundError("BM 통합_데이터 변환_*.xlsx 파일을 찾을 수 없습니다.")

# 최신 파일 선택
matched_files.sort(
    key=lambda f: os.path.getmtime(os.path.join(base_path, f)),
    reverse=True
)

input_file = matched_files[0]
file_path = os.path.join(base_path, input_file)
print(f"📂 입력 파일: {file_path}")

# 파일명에서 기간 suffix 추출 (예: 2025.07~12 / 2025.12)
suffix = input_file.replace("BM 통합_데이터 변환_", "").replace(".xlsx", "")

# =========================
# 2️⃣ 파일 로드
# =========================
df = pd.read_excel(file_path)

# =========================
# 3️⃣ BM코드 매핑 딕셔너리
# =========================
bm_map = {
    "Barclays Global Aggregate (USD Hedged)": "ZCXK000002",
    "Bloomberg Barclays Global Aggregate - Corp(USD Hedged)": "ZCXK000002",
    "MSCI ACWI NR USD": "ZCXK000001",
    "MSCI EM NR USD": "ZDHK000002",
    "MSCI USA NR USD": "ZDHK000003",
    "MSCI WI NR USD": "ZDHK000001"
}

# =========================
# 4️⃣ BM코드 컬럼 생성 (ASSET_ID 기준)
# =========================
df["BM코드"] = df["자산구분(ASSET_ID)"].map(bm_map)

# =========================
# 5️⃣ 데이터ID(DATA_ID) 없는 행 제거
# =========================
df = df.dropna(subset=["데이터ID(DATA_ID)"])

# =========================
# 6️⃣ 결과 확인
# =========================
print(df.head(10))

# =========================
# 7️⃣ 저장 (기간 표기 유지)
# =========================
output_name = f"BM 통합_데이터 변환_BM코드 매핑_{suffix}.xlsx"
output_path = os.path.join(base_path, output_name)

df.to_excel(output_path, index=False)
print(f"✅ {output_name} 저장 완료: {output_path}")


📂 입력 파일: C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\BM 통합_데이터 변환_2025.10~12.xlsx
        기준일    구분  기관코드  펀드코드        BM코드    자산구분(ASSET_ID)  \
0  20251001  FUND  1061   NaN  ZCXK000001  MSCI ACWI NR USD   
1  20251001  FUND  1061   NaN  ZCXK000001  MSCI ACWI NR USD   
2  20251001  FUND  1061   NaN  ZCXK000001  MSCI ACWI NR USD   
3  20251001  FUND  1061   NaN  ZCXK000001  MSCI ACWI NR USD   
4  20251001  FUND  1061   NaN  ZCXK000001  MSCI ACWI NR USD   
5  20251001  FUND  1061   NaN  ZCXK000001  MSCI ACWI NR USD   
6  20251001  FUND  1061   NaN  ZCXK000001  MSCI ACWI NR USD   
7  20251001  FUND  1061   NaN  ZCXK000001  MSCI ACWI NR USD   
8  20251001  FUND  1061   NaN  ZCXK000001  MSCI ACWI NR USD   
9  20251001  FUND  1061   NaN  ZCXK000001  MSCI ACWI NR USD   

           데이터ID(DATA_ID) 데이터구분  비중(WGT)  
0              Financials    BM    16.81  
1  Information Technology    BM    28.51  
2  Consumer Discretionary    BM    10.52  
3             Health Care    BM    

### ! 자산구분 컬럼 변경

In [6]:
import pandas as pd
import os
import re

# =====================================================
# 1️⃣ BM코드 매핑 완료 파일 자동 선택
# =====================================================
base_path = r"C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종"

pattern = re.compile(r"^BM 통합_데이터 변환_BM코드 매핑_.*\.xlsx$")
matched_files = [f for f in os.listdir(base_path) if pattern.match(f)]

if not matched_files:
    raise FileNotFoundError("BM 통합_데이터 변환_BM코드 매핑_*.xlsx 파일을 찾을 수 없습니다.")

matched_files.sort(
    key=lambda f: os.path.getmtime(os.path.join(base_path, f)),
    reverse=True
)

input_file = matched_files[0]
file_path = os.path.join(base_path, input_file)
print(f"📂 입력 파일: {file_path}")

suffix = input_file.replace("BM 통합_데이터 변환_BM코드 매핑_", "").replace(".xlsx", "")

# =====================================================
# 2️⃣ 파일 로드
# =====================================================
df = pd.read_excel(file_path)

# =====================================================
# 3️⃣ 매핑 리스트 정의
# =====================================================
equity_sector = [
    "Financials", "Information Technology", "Consumer Discretionary",
    "Health Care", "Industrials", "Consumer Staples", "Energy",
    "Materials", "Utilities", "Communication Services", "Real Estate"
]

equity_region1 = ["Developed", "Emerging"]

equity_region2 = [
    "North America", "Europe", "Japan", "Asia Pacific",
    "Latin America", "Africa/Middle East", "Others"
]

fixed_region1 = ["Developed", "Emerging"]
fixed_region2 = [
    "North America", "Europe", "Japan", "Asia Pacific",
    "Latin America", "Africa/Middle East", "Others"
]

fixed_maturity = ["0-3Y", "3-5Y", "5-10Y", ">10Y", "Others"]
fixed_rating = ["AAA", "AA", "A", "BBB", "<BBB", "NR"]
fixed_sector = ["Treasury", "Government", "Corporate", "Securitized", "ETC"]

bm_equity = ["ZCXK000001", "ZDHK000002", "ZDHK000003", "ZDHK000001"]
bm_fixed = ["ZCXK000002"]

# =====================================================
# 4️⃣ ASSET_ID 분류 함수
# =====================================================
def get_asset_id(bm, data_id):
    if pd.isna(bm) or pd.isna(data_id):
        return None

    # ---------- Equity ----------
    if bm in bm_equity:
        if data_id in equity_sector:
            return "Equity_Sector"
        elif data_id in equity_region1:
            return "Equity_Region1"
        elif data_id in equity_region2:
            return "Equity_Region2"
        else:
            return "Equity_Country"

    # ---------- Fixed Income ----------
    elif bm in bm_fixed:
        if data_id in fixed_region1:
            return "Fixed Income_Region1"
        elif data_id in fixed_region2:
            return "Fixed Income_Region2"
        elif data_id in fixed_maturity:
            return "Fixed Income_만기"
        elif data_id in fixed_rating:
            return "Fixed Income_신용등급"
        elif data_id in fixed_sector:
            return "Fixed Income_섹터"
        else:
            return "Fixed Income_Country"

    return None

# =====================================================
# 5️⃣ ASSET_ID 적용
# =====================================================
df["자산구분(ASSET_ID)"] = df.apply(
    lambda x: get_asset_id(x["BM코드"], x["데이터ID(DATA_ID)"]),
    axis=1
)

# =====================================================
# 6️⃣ 결과 확인
# =====================================================
print(df[["BM코드", "데이터ID(DATA_ID)", "자산구분(ASSET_ID)"]].head(20))

# =====================================================
# 7️⃣ 저장
# =====================================================
output_name = f"BM 통합_데이터 변환_ASSET_ID 분류완료_{suffix}.xlsx"
output_path = os.path.join(base_path, output_name)

df.to_excel(output_path, index=False)
print(f"✅ 저장 완료: {output_path}")


📂 입력 파일: C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\BM 통합_데이터 변환_BM코드 매핑_2025.10~12.xlsx
          BM코드          데이터ID(DATA_ID)  자산구분(ASSET_ID)
0   ZCXK000001              Financials   Equity_Sector
1   ZCXK000001  Information Technology   Equity_Sector
2   ZCXK000001  Consumer Discretionary   Equity_Sector
3   ZCXK000001             Health Care   Equity_Sector
4   ZCXK000001             Industrials   Equity_Sector
5   ZCXK000001        Consumer Staples   Equity_Sector
6   ZCXK000001                  Energy   Equity_Sector
7   ZCXK000001               Materials   Equity_Sector
8   ZCXK000001               Utilities   Equity_Sector
9   ZCXK000001  Communication Services   Equity_Sector
10  ZCXK000001             Real Estate   Equity_Sector
11  ZCXK000001               Developed  Equity_Region1
12  ZCXK000001                Emerging  Equity_Region1
13  ZCXK000001           North America  Equity_Region2
14  ZCXK000001                  Europe  Equity_Region2
15  ZCXK00000

In [7]:
# =====================================================
# 1️⃣ 중복 순번 생성
#   기준일 + BM코드 + 데이터ID 단위로 중복 순번
# =====================================================
df["dup_rank"] = (
    df
    .groupby(["기준일", "BM코드", "데이터ID(DATA_ID)"])
    .cumcount()
    + 1
)

# =====================================================
# 2️⃣ ASSET_ID 분류 함수 (Japan 중복 예외 포함)
# =====================================================
def get_asset_id(bm, data_id, dup_rank):
    # ---------------------
    # 🔥 Japan 중복 처리
    # 동일 기준일 + 동일 BM코드 + Japan 2번째 이상
    # ---------------------
    if data_id == "Japan" and dup_rank > 1:
        if bm in bm_equity:
            return "Equity_Country"
        elif bm in bm_fixed:
            return "Fixed Income_Country"
        else:
            return None

    # ---------------------
    # 🔽 기존 정상 로직
    # ---------------------
    # ---------- Equity ----------
    if bm in bm_equity:
        if data_id in equity_sector:
            return "Equity_Sector"
        elif data_id in equity_region1:
            return "Equity_Region1"
        elif data_id in equity_region2:
            return "Equity_Region2"
        else:
            return "Equity_Country"

    # ---------- Fixed Income ----------
    elif bm in bm_fixed:
        if data_id in fixed_region1:
            return "Fixed Income_Region1"
        elif data_id in fixed_region2:
            return "Fixed Income_Region2"
        elif data_id in fixed_maturity:
            return "Fixed Income_만기"
        elif data_id in fixed_rating:
            return "Fixed Income_신용등급"
        elif data_id in fixed_sector:
            return "Fixed Income_섹터"
        else:
            return "Fixed Income_Country"

    return None

# =====================================================
# 3️⃣ ASSET_ID 재적용
# =====================================================
df["자산구분(ASSET_ID)"] = df.apply(
    lambda x: get_asset_id(
        x["BM코드"],
        x["데이터ID(DATA_ID)"],
        x["dup_rank"]
    ),
    axis=1
)

# =====================================================
# 4️⃣ 임시 컬럼 제거
# =====================================================
df = df.drop(columns=["dup_rank"])

# =====================================================
# 5️⃣ 결과 확인 (Japan 케이스 중심)
# =====================================================
print(
    df[df["데이터ID(DATA_ID)"] == "Japan"]
    .sort_values(["기준일", "BM코드"])
    [["기준일", "BM코드", "데이터ID(DATA_ID)", "자산구분(ASSET_ID)"]]
    .head(20)
)


          기준일        BM코드 데이터ID(DATA_ID)        자산구분(ASSET_ID)
15   20251001  ZCXK000001          Japan        Equity_Region2
42   20251001  ZCXK000001          Japan        Equity_Country
183  20251001  ZCXK000002          Japan  Fixed Income_Region2
214  20251001  ZCXK000002          Japan  Fixed Income_Country
96   20251001  ZDHK000001          Japan        Equity_Region2
111  20251001  ZDHK000001          Japan        Equity_Country
264  20251101  ZCXK000001          Japan        Equity_Region2
291  20251101  ZCXK000001          Japan        Equity_Country
432  20251101  ZCXK000002          Japan  Fixed Income_Region2
463  20251101  ZCXK000002          Japan  Fixed Income_Country
345  20251101  ZDHK000001          Japan        Equity_Region2
360  20251101  ZDHK000001          Japan        Equity_Country
513  20251201  ZCXK000001          Japan        Equity_Region2
540  20251201  ZCXK000001          Japan        Equity_Country
666  20251201  ZCXK000002          Japan  Fixed Income_

In [8]:
# japan 하나는 Region에 하나는 Country에 반영되었는지 확인

japan_df = df[df["데이터ID(DATA_ID)"] == "Japan"][["기준일", "BM코드", "자산구분(ASSET_ID)", "비중(WGT)"]]


# 모든 행 다 보기
pd.set_option("display.max_rows", None)

japan_df

,기준일,BM코드,자산구분(ASSET_ID),비중(WGT)
15,20251001,ZCXK000001,Equity_Region2,4.891561
42,20251001,ZCXK000001,Equity_Country,4.891561
96,20251001,ZDHK000001,Equity_Region2,5.490000
111,20251001,ZDHK000001,Equity_Country,5.490000
183,20251001,ZCXK000002,Fixed Income_Region2,8.730000
214,20251001,ZCXK000002,Fixed Income_Country,8.730000
264,20251101,ZCXK000001,Equity_Region2,4.881059
291,20251101,ZCXK000001,Equity_Country,4.881059
345,20251101,ZDHK000001,Equity_Region2,5.470000
360,20251101,ZDHK000001,Equity_Country,5.470000


In [14]:
from datetime import datetime

# =====================================================
# 기준일 → 연월 범위 문자열 생성 (연도跨越 대응)
# =====================================================
if "기준일" in df.columns:
    기준일_series = df["기준일"].dropna()

    # YYYYMMDD 숫자형 대응
    if pd.api.types.is_numeric_dtype(기준일_series):
        기준일_dt = pd.to_datetime(
            기준일_series.astype(int).astype(str),
            format="%Y%m%d",
            errors="coerce"
        )
    else:
        기준일_dt = pd.to_datetime(
            기준일_series,
            errors="coerce"
        )

    기준일_dt = 기준일_dt.dropna()

    min_dt = 기준일_dt.min()
    max_dt = 기준일_dt.max()

    min_year = min_dt.strftime("%Y")
    max_year = max_dt.strftime("%Y")

    min_year_2 = min_dt.strftime("%y")
    max_year_2 = max_dt.strftime("%y")

    min_month = min_dt.strftime("%m")
    max_month = max_dt.strftime("%m")

    # ---------------------
    # 표기 규칙
    # ---------------------
    if min_year == max_year:
        if min_month == max_month:
            period_str = f"{min_year}.{max_month}"
        else:
            period_str = f"{min_year}.{min_month}~{max_month}"
    else:
        period_str = f"{min_year_2}.{min_month}~{max_year_2}.{max_month}"

else:
    period_str = datetime.today().strftime("%Y.%m")

# =====================================================
# 저장
# =====================================================
output_name = f"BM 데이터_사학연금({period_str}).xlsx"
output_path = os.path.join(base_path, output_name)

df.to_excel(output_path, index=False)
print(f"✅ 저장 완료: {output_path}")


✅ 저장 완료: C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\BM 데이터_사학연금(2025.10~12).xlsx
